# Графовые нейронные сети

Материалы:
* Макрушин С.В. Курс "Машинное обучение на графах", Лекции 4-5 "Графовые нейронные сети"
* Документация:
    * https://docs.dgl.ai/en/0.6.x/tutorials/blitz/2_dglgraph.html#sphx-glr-tutorials-blitz-2-dglgraph-py
    * https://docs.dgl.ai/en/0.6.x/tutorials/blitz/1_introduction.html#sphx-glr-tutorials-blitz-1-introduction-py
    * https://docs.dgl.ai/generated/dgl.nn.pytorch.conv.GraphConv.html#dgl.nn.pytorch.conv.GraphConv
    * https://docs.dgl.ai/api/python/dgl.data.html
    
Настройка окружения (Google Colab)

```
!pip install dgl
```

## Работа с графами в dgl

In [2]:
!pip install dgl

     |████████████████████████████████| 4.4 MB 5.1 MB/s 


1.1 Cоздайте граф "путь", состоящий из 5 вершин, и представьте его в виде `dgl.graph`. Создайте двумерный тензор размера 5х2 и сохраните его в качестве атрибутов узлов `features`. Выведите основные характеристики графа: количество узлов, количество ребер, размерность признаков узлов. Выведите входящую и исходящую степень узлов.

In [3]:
import dgl
import torch

DGL backend not selected or invalid.  Assuming PyTorch for now.


Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


Using backend: pytorch


In [4]:
g = dgl.graph(
    (
        (0,1,2,3,1,2),
        (1,2,3,4,0,1)
    )
)
g.ndata['feat'] = torch.rand(5,2)
g

Graph(num_nodes=5, num_edges=6,
      ndata_schemes={'feat': Scheme(shape=(2,), dtype=torch.float32)}
      edata_schemes={})

In [9]:
g.number_of_nodes()#колво узлов

5

In [11]:
g.number_of_edges()#колво ребер

6

In [17]:
g.ndata['feat'].shape[1]#размерность признаков узлов

2

In [13]:
g.in_degrees()#входящая степень узлов

tensor([1, 2, 1, 1, 1])

In [14]:
g.out_degrees()#исходящая степень узлов

tensor([1, 2, 2, 1, 0])

1.2 Загрузите граф "Карате Клуб" из `networkx.` Закодируйте значения атрибута `club` на узлах числами 0 (для клуба "Mr. Hi") и 1 (для клуба "Officer"). Преобразуйте его к графу `dgl.graph` при помощи `dgl.from_networkx`, сохранив значения атрибута узлов `club`. Добавьте в качестве атрибутов узлов единичную матрицу размера 34х34. Выведите основные характеристики графа: количество узлов, количество ребер, размерность признаков узлов, кол-во уникальных классов узлов.

1.3 Загрузите датасет `CoraFullDataset` из `dgl.data`. Этот датасет состоит из одного графа. Выведите основные характеристики графа: количество узлов, количество ребер, размерность признаков узлов, кол-во уникальных классов узлов.

In [18]:
data = dgl.data.CoraFullDataset()
g = data[0]

Extracting file to /root/.dgl/cora_full


In [19]:
g.number_of_nodes()#колво узлов

19793

In [20]:
g.number_of_edges()#колво ребер

126842

In [21]:
g.ndata['feat'].shape[1]#размерность признаков узлов

8710

In [25]:
len(g.ndata['label'].unique())#колво уникальных классов

70

1.4 Для графа `CoraFullDataset` выделите подграф (`dgl.node_subgraph`), содержащий узлы, относящиеся к трем наиболее часто встречающимся классам.

## Построение нейронных сетей с использованием `dgl`

2.1 Загрузите граф Cora из файла `CoraGraphDataset.dgl.pickle`. Решите задачу классификации узлов графа, используя только полносвязные слои `torch.nn.Linear`. Обратите внимание, что настройка весов модели должна проводиться только на основе примеров из обучающей выборки. Посчитайте значение `accuracy` на тестовой выборке.

In [41]:
import torch.nn as nn
import torch.optim as optim

In [42]:
import pickle
with open('CoraGraphDataset.dgl.pickle','rb') as f:
  g=pickle.load(f)

In [43]:
g

Graph(num_nodes=2708, num_edges=10556,
      ndata_schemes={'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool), 'label': Scheme(shape=(), dtype=torch.int64), 'feat': Scheme(shape=(1433,), dtype=torch.float32)}
      edata_schemes={'__orig__': Scheme(shape=(), dtype=torch.int64)})

In [44]:
g.ndata['label'].unique()

tensor([0, 1, 2, 3, 4, 5, 6])

In [45]:
train=g.ndata['train_mask']
test=g.ndata['test_mask']
val=g.ndata['val_mask']

In [46]:
# 1. Определить модель
class TwoLayersNet(nn.Module):
    def __init__(self, nX, nH, nY):        
        super(TwoLayersNet, self).__init__()     
         
        self.fc1 = nn.Linear(nX, nH)            
        self.fc2 = nn.Linear(nH, nY)             
          
    def forward(self, x):                        
        x = self.fc1(x)                          
        x = nn.ReLU()(x)                      
        x = self.fc2(x)                         
        return x

model = TwoLayersNet(1433,16,7)
X=g.ndata['feat']
y=g.ndata['label']
#2. Определиться с функцией потерь
criterion = nn.CrossEntropyLoss()

# 3. Определяемся с методом оптимизации
optimizer = torch.optim.Adam(model.parameters(),          
                            lr=0.01)

# 4. Процесс обучения
for i in range(100):
  y_pred = model(X[train])
  loss = criterion(y_pred,y[train])
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f'{i} {loss.item()}')

0 1.955780029296875
1 1.9507275819778442
2 1.9450457096099854
3 1.938631296157837
4 1.931694507598877
5 1.9241877794265747
6 1.9159188270568848
7 1.9068448543548584
8 1.896972417831421
9 1.8862910270690918
10 1.8747689723968506
11 1.8623967170715332
12 1.8491178750991821
13 1.8349114656448364
14 1.8197470903396606
15 1.8035904169082642
16 1.7864506244659424
17 1.76829195022583
18 1.7491188049316406
19 1.7289128303527832
20 1.7076746225357056
21 1.6853975057601929
22 1.6620670557022095
23 1.6377354860305786
24 1.6124351024627686
25 1.5861397981643677
26 1.5589102506637573
27 1.5307483673095703
28 1.501708745956421
29 1.4718700647354126
30 1.4412333965301514
31 1.4098834991455078
32 1.3778306245803833
33 1.3451805114746094
34 1.3119696378707886
35 1.2782604694366455
36 1.2441449165344238
37 1.2096856832504272
38 1.174992322921753
39 1.1401450634002686
40 1.1051747798919678
41 1.070152997970581
42 1.035197138786316
43 1.0003621578216553
44 0.9657130837440491
45 0.9313628077507019
46 0.897

In [47]:
from sklearn.metrics import accuracy_score

In [48]:
y_pred = model(X[test])
y_class=torch.argmax(y_pred,axis=1)

In [49]:
accuracy_score(y[test],y_class)

0.469

2.2 Повторите решение задачи 2.1, используя один графовый сверточный слой.

In [56]:
# 1. Определить модель
class TwoLayersNet(nn.Module):
    def __init__(self, nX, nH):        
        super(TwoLayersNet, self).__init__()     
         
        self.conv1 = dgl.nn.GraphConv(nX, nH)
          
    def forward(self, g, x):                        
        x = self.conv1(g, x)
        return x

model = TwoLayersNet(1433,7)
X=g.ndata['feat']
y=g.ndata['label']
#2. Определиться с функцией потерь
criterion = nn.CrossEntropyLoss()

# 3. Определяемся с методом оптимизации
optimizer = torch.optim.Adam(model.parameters(),         
                            lr=0.01)

# 4. Процесс обучения
for i in range(100):
  y_pred = model(g, X)
  loss = criterion(y_pred[train],y[train])
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f'{i} {loss.item()}')

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0 1.9452520608901978
1 1.9369977712631226
2 1.9287220239639282
3 1.9204812049865723
4 1.912264108657837
5 1.9040582180023193
6 1.8958666324615479
7 1.8876971006393433
8 1.879550814628601
9 1.871423363685608
10 1.8633135557174683
11 1.855223298072815
12 1.8471542596817017
13 1.839106559753418
14 1.8310807943344116
15 1.8230769634246826
16 1.8150947093963623
17 1.8071346282958984
18 1.7991971969604492
19 1.7912832498550415
20 1.7833929061889648
21 1.775526523590088
22 1.7676838636398315
23 1.7598652839660645
24 1.7520713806152344
25 1.74430251121521
26 1.7365589141845703
27 1.7288405895233154
28 1.721148133277893
29 1.7134819030761719
30 1.7058415412902832
31 1.6982274055480957
32 1.6906399726867676
33 1.683079719543457
34 1.6755465269088745
35 1.668040156364441
36 1.660561203956604
37 1.653110146522522
38 1.6456868648529053
39 1.638291597366333
40 1.6309244632720947
41 1.6235853433609009
42 1.6162748336791992
43 1.6089926958084106
44 1.601739525794983
45 1.5945148468017578
46 1.58731937

In [58]:
y_pred = model(g, X)
y_class=torch.argmax(y_pred[test],axis=1)

In [59]:
accuracy_score(y[test],y_class)

0.659

2.3. Перепишите решение задачи 2.2, используя два слоя `dgl.nn.GraphConv`.

In [60]:
# 1. Определить модель
class TwoLayersNet(nn.Module):
    def __init__(self, nX, nH, nY):        
        super(TwoLayersNet, self).__init__()     
         
        self.conv1 = dgl.nn.GraphConv(nX, nH)
        self.conv2 = dgl.nn.GraphConv(nH, nY)             
          
    def forward(self, g, x):                        
        x = self.conv1(g, x)
        x = nn.ReLU()(x)                     
        x = self.conv2(g, x)                  
        return x

model = TwoLayersNet(1433,16,7)
X=g.ndata['feat']
y=g.ndata['label']
#2. Определиться с функцией потерь
criterion = nn.CrossEntropyLoss()

# 3. Определяемся с методом оптимизации
optimizer = torch.optim.Adam(model.parameters(),         
                            lr=0.01)

# 4. Процесс обучения
for i in range(100):
  y_pred = model(g, X)
  loss = criterion(y_pred[train],y[train])
  loss.backward()
  optimizer.step()
  optimizer.zero_grad()
  print(f'{i} {loss.item()}')

/usr/local/lib/python3.7/dist-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


0 1.9456390142440796
1 1.938171625137329
2 1.92892587184906
3 1.9172797203063965
4 1.9050045013427734
5 1.8926459550857544
6 1.8791379928588867
7 1.8641237020492554
8 1.8472284078598022
9 1.8290361166000366
10 1.8106070756912231
11 1.7917389869689941
12 1.771891474723816
13 1.7509651184082031
14 1.7289413213729858
15 1.7058483362197876
16 1.6817214488983154
17 1.656624674797058
18 1.6305785179138184
19 1.603597640991211
20 1.5757204294204712
21 1.5469729900360107
22 1.5173927545547485
23 1.4870058298110962
24 1.455880045890808
25 1.4240785837173462
26 1.3916388750076294
27 1.3585872650146484
28 1.325000524520874
29 1.2909636497497559
30 1.2565280199050903
31 1.2217512130737305
32 1.1867496967315674
33 1.1515835523605347
34 1.1163442134857178
35 1.081117033958435
36 1.0459786653518677
37 1.01101553440094
38 0.9763060212135315
39 0.941926121711731
40 0.9079540371894836
41 0.8744661211967468
42 0.8415271639823914
43 0.8092070817947388
44 0.7775614857673645
45 0.7466419339179993
46 0.71650

In [61]:
y_pred = model(g, X)
y_class=torch.argmax(y_pred[test],axis=1)

In [62]:
accuracy_score(y[test],y_class)

0.771